L'objectif de ce projet est d'observer s'il existe des "déserts sportifs", 
lieux en France où les infrastrcutures sportives manquent. 
La question sera alors d'essayer d'expliquer ces déserts sportifs, que ce soit par des raisons économiques ou politiques. 
Enfin, il s'agira de comparer la carte des déserts sportifs avec d'autres cartes connues, à l'instar des déserts médicaux.

In [ ]:
#Installation des modules
!pip install py7zr geopandas openpyxl tqdm s3fs --quiet
!pip install PyYAML xlrd --quiet
!pip install cartiflette --quiet
!pip install contextily --quiet
!pip install folium --quiet
!pip install nb-clean --quiet

In [ ]:
#Importation des modules
import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download
import requests
import io
import zipfile
import folium
from folium.plugins import HeatMap



In [ ]:
#Importation du jeu de données principal (localisation des équipements sportifs en France)

url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)



#Importation d'un jeu de données csv sur des informations économiques et démographiques au niveau des communes

urlpop = "https://www.insee.fr/fr/statistiques/fichier/2521169/base_cc_comparateur_csv.zip"
response = requests.get(urlpop)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:                 #io.BytesIO évite de télécharger le fichier
    with z.open("base_cc_comparateur.csv") as csvfile:                   #sur le disque dur
        df_communes = pd.read_csv(csvfile, sep=";", low_memory=False)


#Importation d'un jeu de données politiques au niveau des communes: résultats des législatives 2024 (2nd tour)

url2="https://www.data.gouv.fr/api/1/datasets/r/5a8088fd-8168-402a-9f40-c48daab88cd1"
legislatives2=pd.read_csv(url2, sep=";", low_memory=False)

In [ ]:
#Visualisation de la base de données principale
equipement.head(3)

In [ ]:
#Visualisation de la base de données sur les informations économiques et démographiques
df_communes.head(3)

In [ ]:
#Visualisation de la base de données politique
legislatives2.head(3)

In [ ]:
#On conserve les variables qui nous intéressent pour chaque jeu de données

#Pour le jeu de données principal
cols = ["Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Densite Catégorie", "Type d'équipement sportif", 
        "Longitude", "Latitude"]
equipement = equipement[cols]

#Pour le jeu de données sur les informations économiques et démographiques
cols = ["CODGEO", "P22_POP", "P22_MEN","MED21", "TP6021", "P22_CHOM1564"]
df_communes = df_communes[cols]

#Pour le jeu de données politiques
cols = ["Code commune", "Libellé commune"]
cols += [(f"Nuance candidat {i}") for i in range(1, 5)]
cols += [(f"Elu {i}") for i in range(1, 5)]

legislatives2 = legislatives2[cols]

#Transformation de la variable élu en binaire pour tous les candidats
elu_cols = [f"Elu {i}" for i in range(1, 5)] 
for col in elu_cols:
    legislatives2[col] = legislatives2[col].notna().astype(int)  #Transforme les booléens en valeur 0 ou 1

In [ ]:
#Visualisation nouvelle base de données principale
equipement.head(3)

In [ ]:
#Visualisation nouvelle base de données sur les informations économiques et démographiques
df_communes.head(3)

In [ ]:
#Visualisation nouvelle base de données politique
legislatives2.head(3)

In [ ]:
#Dictionnaire des variables utilisées dans df_commune
pd.set_option("display.max_colwidth", None)

tab_communes = {
    "Variable": [
        "CODGEO", "P22_POP", "P22_MEN", "MED21", "TP6021", "P22_CHOM1564"
    ],
    "Description": [
        "Code INSEE de la commune",
        "Population en 2022",
        "Nombre de ménages en 2022",
        "Médiane du niveau de vie en 2021",
        "Taux de pauvreté en 2021",
        "Nombre de chômeurs de 15 à 64 ans en 2022"
    ]
}

dico_vars_communes = pd.DataFrame(tab_communes)
dico_vars_communes

In [ ]:
#Dictionnaire des variables utilisées dans legislatives


tab_pol = {
    "Variable": ["Code commune", "Libellé commune", "Nuance candidat", "Elu"],
    "Description": [
        "Code INSEE de la commune",
        "Nom de la commune",
        "Parti politique du candidat",
        "Variable binaire égale à 1 si le candidat a été élu, 0 sinon"
    ]
}

dico_vars_pol = pd.DataFrame(tab_pol)
dico_vars_pol

In [ ]:
#On souhaite créer une variable "Nuance politique du candidat élu" pour notre base de données finale

def get_nuance_elu(row):
    """
    Retourne la nuance du candidat élu pour la commune de la ligne "row"
    """
    if row['Elu 1']:
        return row['Nuance candidat 1']
    elif row['Elu 2']:
        return row['Nuance candidat 2']
    elif row['Elu 3']:
        return row['Nuance candidat 3']
    elif row['Elu 4']:
        return row['Nuance candidat 4']
    else:
        return np.nan

legislatives2['Nuance candidat élu'] = legislatives2.apply(get_nuance_elu, axis=1)

In [ ]:
legislatives2.head(3)

In [ ]:
#On garde seulement la nuance du candidat élu, ce qui nous intéresse ici
cols = ["Code commune", "Nuance candidat élu"]
legislatives2 = legislatives2[cols]
legislatives2.head(3)

In [ ]:
#On réunit tous les jeux de données pour obtenir notre jeu de données final

df_communes = df_communes.rename(columns={"CODGEO": "Commune INSEE"})
legislatives2 = legislatives2.rename(columns={"Code commune": "Commune INSEE"}) #On renomme les colonnes pour concaténer

df_final = (
    equipement
    .merge(df_communes, on="Commune INSEE", how="left")
    .merge(legislatives2, on="Commune INSEE", how="left")
)

df_final.head(12)

In [ ]:
#En faisant le test avec ma commune d'origine, on se rend compte que des lignes sont parfois en double, 
#voire triple, on va donc supprimer ces doublons.

test=df_final[df_final["Commune nom"] == "Eschau"]
test



In [ ]:
#Supression des lignes doublons
df_final = df_final.drop_duplicates()

In [ ]:
#Maintenant, on va regarder le type des colonnes pour mettre les variables dans le format
#qui nous arrange

print(df_final.dtypes)

In [ ]:
#On va convertir les Code de départements en entier

#On fait le choix de Remplacer 2A et 2B par 96 et 97
df_final['Département Code'] = df_final['Département Code'].replace({'2A': '96', '2B': '97'})

#On convertit en entier, coerce nous permet de gérer les erreurs en cas de valeur manquante
df_final['Département Code'] = pd.to_numeric(df_final['Département Code'], errors='coerce').astype('Int64')

In [ ]:
#Finalement, on ne conserve que les données sur la France métropolitaine, afin de faciliter les 
#représentations graphiques (cartes)

df_final = df_final[1 <= (df_final['Département Code'] <= 97)]

In [ ]:
df_final['MED21'] = pd.to_numeric(df_final['MED21'], errors='coerce').astype('Int64')


On va maintenant passer à une représentation graphique des données


In [ ]:
#On récupère le fond de carte de la france métropolitaine


#Téléchargement de toute la France avec les DROM
gdf = carti_download(
    values="France",
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

#Filtrer pour ne garder que la métropole (exclure les DROM)
departements_drom = ["971", "972", "973", "974", "975", "976"]
gdf = gdf[~gdf['INSEE_DEP'].isin(departements_drom)]

In [ ]:
gdf_pts = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final["Longitude"], df_final["Latitude"]),
    crs="EPSG:4326"
)


fig, ax = plt.subplots(figsize=(20, 10))
gdf.boundary.plot(ax=ax, linewidth=0.4, color="gray")
gdf_pts.plot(ax=ax, markersize=3, alpha=0.7, color="red")
ax.set_xlim(-5.5, 10) 
ax.set_ylim(41, 51)
ax.set_title("Infrastructures sportives — France", fontsize=14)
ax.set_axis_off()
plt.savefig('docs/premiere_carte.png', dpi=300, bbox_inches='tight')

On voit que le schéma de la "diagonale du vide" semble se reproduire avec les infrastructures sportives.
La densité des infrastructures sportives en Corse semble moins importante qu'en France métropolitaine.

In [ ]:
#Carte de chaleur 


df = df_final.copy().dropna(subset=['Latitude', 'Longitude'])  #Supression des valeurs manquantes

#Création de la carte, centrée sur la France
calor = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles='CartoDB positron'
)

#Récupération des données géographiques
heat_data = [[row['Latitude'], row['Longitude']] for idx, row in df.iterrows()]


#On complète la carte
HeatMap(
    heat_data,
    radius=15,           # Rayon de chaque point
    blur=20,             # Flou pour adoucir
    max_zoom=13,         # Zoom maximum
    gradient={            # Dégradé de couleurs personnalisé
        0.0: 'blue',
        0.3: 'lime',
        0.5: 'yellow',
        0.7: 'orange',
        1.0: 'red'
    }
).add_to(calor)
calor.save('docs/heatmap_toutes_infrastructures.html') #On sauvegarde la carte

In [ ]:
#On crée une carte qui compte le nombre d'infrastructures sportives par département

infra_par_dept = df_final.copy().groupby('Département Code').size().reset_index(name='Nombre_infrastructures')

#On réadapte les codes de département pour correspondre au fond de carte
infra_par_dept['Département Code'] = infra_par_dept['Département Code'].astype(str)
infra_par_dept['Département Code'] = infra_par_dept['Département Code'].replace({
    '96': '2A',
    '97': '2B'
})

def format_dept_code(code):
    """
    Ajouter un zéro devant les codes à 1 chiffre (1 → 01)
    """
    if code in ['2A', '2B']:
        return code
    if int(code) >= 10:
        return code
    else:
        return "0" + code

infra_par_dept['Département Code'] = infra_par_dept['Département Code'].apply(format_dept_code)
print("-" * 50)
print("Visualisation de la base de données infra_par_dept")
print("-" * 50)
print(infra_par_dept.head(4))


gdf2 = gdf.copy() #On copie le fond de carte pour ne pas le modifier et pouvoir le réutiliser


gdf2 = gdf2.merge(               #Fusion des données
    infra_par_dept, 
    left_on='INSEE_DEP', 
    right_on='Département Code', 
    how='left'
)                 


#Création de la carte, centrée sur la France
m = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles='CartoDB positron'
)

#On complète la carte
folium.Choropleth(
    geo_data=gdf2,
    name='choropleth',
    data=infra_par_dept,
    columns=['Département Code', 'Nombre_infrastructures'],
    key_on='feature.properties.INSEE_DEP',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Nombre d\'infrastructures sportives par département'
).add_to(m)

#Ajout des informations au survol de la souris
folium.GeoJson(
    gdf2,
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
    tooltip=folium.GeoJsonTooltip(
        fields=['INSEE_DEP', 'Nombre_infrastructures'],
        aliases=['Département:', 'Nombre d\'infrastructures:'],
        localize=True
    )
).add_to(m)
m.save('docs/carte_departements.html')

On observe sur cette carte que la Moselle et le Nord sont les départements comptant le plus grand nombre d’infrastructures sportives, tandis que les deux départements corses en disposent nettement moins.
Cependant, cette lecture doit être nuancée : la carte ne tient pas compte du nombre d’habitants ni du revenu médian de chaque département, deux facteurs essentiels pour interpréter correctement ces écarts.
Observons maintenant l'effet de la population sur cette carte.

In [ ]:
#Estimation du nombre d'habitants par département avec les communes disponibles dans la base de données
pop_par_dept = df_final.copy()[['Département Code', 'Commune INSEE', 'P22_POP']].drop_duplicates(
    subset=['Commune INSEE']
).groupby('Département Code')['P22_POP'].sum().reset_index()


pop_par_dept['Département Code'] = pop_par_dept['Département Code'].astype(str)
pop_par_dept['Département Code'] = pop_par_dept['Département Code'].replace({
    '96': '2A',
    '97': '2B'
})

pop_par_dept.columns = ['Département Code', 'Population_totale']
pop_par_dept['Département Code'] = pop_par_dept['Département Code'].apply(format_dept_code)

infra_par_dept = infra_par_dept.merge(pop_par_dept, on='Département Code', how='left')

infra_par_dept['Infra_pour_10k_hab'] = (
    infra_par_dept['Nombre_infrastructures'] / infra_par_dept['Population_totale'] * 10000
).round(2) #Calcul du nombre d'infrastructures pour 10000 habitants

print("-" * 50)
print("Visualisation de la base de données infra_par_dept")
print("-" * 50)
print(infra_par_dept.head(4))

gdf3 = gdf.copy()

gdf3 = gdf3.merge(
    infra_par_dept, 
    left_on='INSEE_DEP', 
    right_on='Département Code', 
    how='left'
)

n = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles='CartoDB positron'
)

folium.Choropleth(
    geo_data=gdf3,
    name='choropleth',
    data=infra_par_dept,
    columns=['Département Code', 'Infra_pour_10k_hab'],
    key_on='feature.properties.INSEE_DEP',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Infrastructures sportives pour 10 000 habitants',
    nan_fill_color='lightgray'
).add_to(n)

folium.GeoJson(
    gdf3,
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
    tooltip=folium.GeoJsonTooltip(
        fields=['INSEE_DEP', 'Nombre_infrastructures', 'Population_totale', 'Infra_pour_10k_hab'],
        aliases=[
            'Département:', 
            'Nombre d\'infrastructures:', 
            'Population totale:',
            'Infrastructures / 10k hab:'
        ],
        localize=True
    )
).add_to(n)
n.save('docs/carte_infra_par_habitant.html')


Le résultat change totalement puisque c'est désormais les Hautes Alpes qui détiennent le plus d'infrastructures sportives par 10 000 habitants.

On va maintenant passer à des statistiques descriptives numériques 

In [ ]:
print(f"Il y a {df_final["Type d'équipement sportif"].count()} infrastructures sportives dans notre base de données")

In [ ]:
print("Voici le type d'infrastructures qu'on retrouve dans notre base de données : \n \n")
df_final.copy()["Type d'équipement sportif"].value_counts().reset_index().head(15)


In [ ]:
infra_par_commune = (
    df_final.copy().groupby("Commune nom")
            .size()
            .reset_index(name="Nombre d'infrastructures")
)

infra_par_commune.describe()

In [ ]:
infra_par_commune.sort_values("Nombre d'infrastructures", ascending=False).head(10)

In [ ]:
arrondissements_paris = df_final.copy()[
    df_final["Commune INSEE"].astype(str).between("75100", "75120")
]

infra_par_arrdt = (
    arrondissements_paris
    .groupby("Commune nom")
    .size()
    .reset_index(name="Nombre d'infrastructures")
    .sort_values("Nombre d'infrastructures", ascending=False)
)

infra_par_arrdt



In [ ]:
infra_par_dept.sort_values("Nombre_infrastructures", ascending=False)

In [ ]:
infra_par_dept.sort_values("Infra_pour_10k_hab", ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(infra_par_commune["Nombre d'infrastructures"], bins=100)
plt.yscale("log")
plt.xlabel("Nombre d'infrastructures par commune")
plt.ylabel("Nombre de communes (échelle log)")
plt.title("Distribution (échelle logarithmique)")
plt.show()




In [ ]:
#Ca ressemble à une loi expo (ptet commenter dessus ?)

In [ ]:
infra_par_densite = (
    df_final.copy().groupby("Densite Catégorie")
            .size()
            .reset_index(name="Nombre d'infrastructures")
)
infra_par_densite = infra_par_densite.sort_values("Nombre d'infrastructures", ascending=False)
infra_par_densite

In [ ]:
df_med = infra_par_commune.merge(
    df_final.copy()[["Commune nom", "MED21"]],
    on= "Commune nom",
    how="left"
)


df_med["Quartile Revenu"] = pd.qcut(
    df_med["MED21"],
    q=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)

infra_par_quartile = (
    df_med.groupby("Quartile Revenu", observed = True)["Nombre d'infrastructures"]
            .mean()
            .reset_index()
)

infra_par_quartile.columns = ["Quartile Revenu", "Nombre d'infrastructures moyen par commune"]
infra_par_quartile

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(infra_par_quartile["Quartile Revenu"], infra_par_quartile["Nombre d'infrastructures moyen par commune"])
plt.xlabel("Quartile de revenu médian (MED21)")
plt.ylabel("Nombre moyen d'infrastructures")
plt.title("Nombre moyen d'infrastructures sportives selon le revenu médian des communes")
plt.show()

In [ ]:
df_chom = infra_par_commune.merge(
    df_final.copy()[["Commune nom", "P22_CHOM1564"]],
    on= "Commune nom",
    how="left"
)


df_chom["Quartile Chômage"] = pd.qcut(
    df_chom["P22_CHOM1564"],
    q=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)

infra_par_quartile_chom = (
    df_chom.groupby("Quartile Chômage", observed = True)["Nombre d'infrastructures"]
            .mean()
            .reset_index()
)

infra_par_quartile_chom.columns = ["Quartile Chômage", "Nombre d'infrastructures moyen par commune"]
infra_par_quartile_chom

In [ ]:
#Bizarre (regarder plus précisemment maybe)

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(infra_par_quartile_chom["Quartile Chômage"], infra_par_quartile_chom["Nombre d'infrastructures moyen par commune"])
plt.xlabel("Quartile du chômage (P22_CHOM1564)")
plt.ylabel("Nombre moyen d'infrastructures")
plt.title("Nombre moyen d'infrastructures sportives selon les quartiles de chômage")
plt.show()

In [ ]:
df_pol = infra_par_commune.merge(
    df_final.copy()[["Commune nom", "Nuance candidat élu"]],
    on= "Commune nom",
    how="left"
)


infra_par_pol= (
    df_pol.groupby("Nuance candidat élu", observed = True)["Nombre d'infrastructures"]
            .mean()
            .reset_index()
)

infra_par_pol.columns = ["Nuance du candidat élu", "Nombre d'infrastructures moyen par commune"]
infra_par_pol = infra_par_pol.sort_values("Nombre d'infrastructures moyen par commune", ascending=False)
infra_par_pol

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(infra_par_pol["Nuance du candidat élu"], infra_par_pol["Nombre d'infrastructures moyen par commune"])
plt.xlabel("Nuance du candidat élu")
plt.ylabel("Nombre moyen d'infrastructures par commune")
plt.title("Nombre moyen d'infrastructures sportives selon la nuance politique des élus")
plt.show()

In [ ]:
#Peut être faire une fonction pour les barplots (code redondant) et même peut être pour les groupby
#précédants mais plus dur

In [ ]:
!nb-clean clean Projet.ipynb

In [ ]:
!ls -lh Projet.ipynb